Extraindo Projeto ETL Ocorrência de crimes.

Abaixo segue tentiva para extrair e tratarmos os dados extraídos de crimes e matrículas nas escolas, mesmo com erros, optamos por manter essa tentativa para demonstrarmos que tentamos realizar o projeto via código, porém continuamos com tratamento via excel e power querry, trazendo essa visão do excel para o Power BI, possibilitando análises de como as matrículas nas escolas influenciam os crimes em SP

In [ ]:
!pip install scrapy

In [1]:
!scrapy startproject crimes_sp

New Scrapy project 'crimes_sp', using template directory 'C:\Users\goula\anaconda3\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\goula\Desktop\Projeto ETL\crimes_sp

You can start your first spider with:
    cd crimes_sp
    scrapy genspider example example.com


In [2]:
%cd crimes_sp

c:\Users\goula\Desktop\Projeto ETL\crimes_sp


In [3]:
!scrapy genspider ssp_sp ssp.sp.gov.br

Created spider 'ssp_sp' using template 'basic' in module:
  crimes_sp.spiders.ssp_sp


Tentativa de automação de processo de extração via scrapy, tivemos dificuldade pelo caminho e filtro de mais de 600 municípios

In [4]:
%%writefile crimes_sp/spiders/ssp_sp.py
import scrapy
import os

class SspSpSpider(scrapy.Spider):
    name = 'ssp_sp'
    allowed_domains = ['ssp.sp.gov.br']
    start_urls = ['https://www.ssp.sp.gov.br/estatistica/dados-mensais']

    def parse(self, response):
        # Encontrar o botão "ocorrências por ano" e seguir o link
        ano_url = response.css('a[href*="ocorrenciasporano"]::attr(href)').get()
        if ano_url:
            yield response.follow(ano_url, self.parse_ocorrencias)

    def parse_ocorrencias(self, response):
        # Iterar sobre a lista de municípios
        municipios = response.css('select[formcontrolname="anualMunicipio"] option::attr(value)').getall()
        municipios = municipios[1:]  # Ignorar a primeira opção "Municípios"
        
        for municipio in municipios:
            municipio_url = f"https://www.ssp.sp.gov.br/estatistica/ocorrenciasporano/{municipio}.aspx"
            yield response.follow(municipio_url, self.parse_municipio)

    def parse_municipio(self, response):
    # Encontrar o botão "Exportar Dados" usando o seletor CSS apropriado
    export_button = response.css('button.btn-export')
    if export_button:
        # Clicar no botão de exportação para iniciar o download
        yield scrapy.FormRequest.from_response(
            response,
            formcss='button.btn-export',
            callback=self.download_data
        )
    else:
        self.log(f'Export button not found for {response.url}', level=scrapy.log.ERROR)

    def download_data(self, response):
        # Extrair o nome do arquivo a partir da URL
        path = response.url.split('/')[-1]
        # Criar um diretório para salvar os arquivos, se não existir
        if not os.path.exists('data'):
            os.makedirs('data')
        # Caminho completo do arquivo
        full_path = os.path.join('data', path)
        self.log(f'Saving file {full_path}')
        # Salvar o conteúdo do arquivo
        with open(full_path, 'wb') as f:
            f.write(response.body)

Overwriting crimes_sp/spiders/ssp_sp.py


In [ ]:
!scrapy crawl ssp_sp

In [ ]:
!pip install pandas openpyxl

Tratando dados de uma planilha por municipio separando por ano

In [ ]:
import pandas as pd

# Data provided by the user
file_path = "C:\Users\goula\Downloads\OcorrenciaAnual-Adamantina_20240525_185440.xlsx"
df = pd.read_excel(file_path)

# removendo colunas
df = df.drop(columns=["homicidio", "frv"])

# separando DataFrame por ano
dfs_by_year = {year: df[df["ano"] == year].copy() for year in df["ano"].unique()}

output_path = "C:\Users\goula\Desktop\Projeto_A3.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for year, data in dfs_by_year.items():
        data.to_excel(writer, sheet_name=str(year), index=False)

print(f"Arquivo salvo com sucesso")

Tratando dados extraídos de Matrículas das Escolas em SP

In [ ]:
import pandas as pd

# Leitura do arquivo enviado
file_path = "C:\Users\goula\Downloads\Censo_escolar_-_sinopse_1 (1).xlsx"
df = pd.read_excel(file_path)

# Remover espaços em branco dos nomes das colunas
df.columns = df.columns.str.strip()

# Filtrando os dados para ter somente dados do enisno médio
df_ensino_medio = df[df['Nome'] == 'Ensino médio']

# Removendo colunas desnecessárias
df_ensino_medio = df_ensino_medio.drop(columns=["Posição", "Nome", "Unidade", "Multiplicador"])

# Criando um DataFrame separado para cada ano e salvando em um arquivo Excel
output_path = "C:\Users\goula\Desktop\Projeto_A3.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for year in range(2008, 2022):
        df_year = df_ensino_medio[["Localidade", str(year)]]
        df_year.columns = ["Municipio", "Matriculas"]
        df_year.to_excel(writer, sheet_name=str(year), index=False)

print(f"Arquivo salvo com sucesso")